In [1]:
%matplotlib inline
import os
from glob import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import geopandas as gpd

In [2]:
data_dir = '/cluster/tufts/hugheslab/datasets/NSF_OD/'
svi_dir = os.path.join(data_dir,'SocialVulnerabilityIndex')
result_dir = os.path.join(data_dir, 'results_20220606_update')
mass_shapefile = os.path.join(data_dir,'shapefiles','MA_2021')

if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [3]:
deaths_file = os.path.join(result_dir,'res_deaths_month_all')
deaths_gdf = gpd.read_file(deaths_file)


In [4]:
deaths_gdf.set_index(['TRACTCE','year','month']).sort_index().loc[pd.IndexSlice[101,:,:],:]

STATEFP COUNTYFP        GEOID  NAME           NAMELSAD  \
TRACTCE year month                                                          
101     2000 1          25      025  25025000101  1.01  Census Tract 1.01   
             2          25      025  25025000101  1.01  Census Tract 1.01   
             3          25      025  25025000101  1.01  Census Tract 1.01   
             4          25      025  25025000101  1.01  Census Tract 1.01   
             5          25      025  25025000101  1.01  Census Tract 1.01   
...                    ...      ...          ...   ...                ...   
        2021 8          25      025  25025000101  1.01  Census Tract 1.01   
             9          25      025  25025000101  1.01  Census Tract 1.01   
             10         25      025  25025000101  1.01  Census Tract 1.01   
             11         25      025  25025000101  1.01  Census Tract 1.01   
             12         25      025  25025000101  1.01  Census Tract 1.01   

                    MTFCC FUNCSTAT   ALAND  AWATER     INTPTLAT      INTPTLON  \
TRACTCE year month                                                              
101     2000 1      G5020        S  249924       0  +42.3608778  -071.1330510   
             2      G5020        S  249924       0  +42.3608778  -071.1330510   
             3      G5020        S  249924       0  +42.3608778  -071.1330510   
             4      G5020        S  249924       0  +42.3608778  -071.1330510   
             5      G5020        S  249924       0  +42.3608778  -071.1330510   
...                   ...      ...     ...     ...          ...           ...   
        2021 8      G5020        S  249924       0  +42.3608778  -071.1330510   
             9      G5020        S  249924       0  +42.3608778  -071.1330510   
             10     G5020        S  249924       0  +42.3608778  -071.1330510   
             11     G5020        S  249924       0  +42.3608778  -071.1330510   
             12     G5020        S  249924       0  +42.3608778  -071.1330510   

                    tract  deaths  \
TRACTCE year month                  
101     2000 1        NaN     0.0   
             2        NaN     0.0   
             3        NaN     0.0   
             4        NaN     0.0   
             5        NaN     0.0   
...                   ...     ...   
        2021 8        NaN     0.0   
             9        NaN     0.0   
             10       NaN     0.0   
             11       NaN     0.0   
             12       NaN     0.0   

                                                             geometry  
TRACTCE year month                                                     
101     2000 1      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             2      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             3      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             4      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             5      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
...                                                               ...  
        2021 8      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             9      POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             10     POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             11     POLYGON ((-71.13660 42.36016, -71.13633 42.360...  
             12     POLYGON ((-71.13660 42.36016, -71.13633 42.360...  

[264 rows x 14 columns]

In [5]:
deaths_gdf.loc[:,'TRACTCE'] = deaths_gdf.loc[:,'TRACTCE'].astype(str)
deaths_gdf.loc[:,'GEOID'] = deaths_gdf.loc[:,'GEOID'].astype(str)


all_df = gpd.GeoDataFrame()
for year in range(2000,2022):
    if year <= 2006:
        svi_year=2000
        theme_cols = ['MAG1TP','MAG2TP','MAG3TP','MAG4TP','MATP']
        tract_col = 'TRACT'
        geo_col = 'FIPS'
    elif year <= 2012:
        svi_year = 2010
        theme_cols = ['R_PL_THEME1','R_PL_THEME2','R_PL_THEME3','R_PL_THEME4','R_PL_THEMES']
        tract_col = 'TRACT'
    elif year <=2014:
        svi_year = 2014
        theme_cols = ['RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES']
        tract_col = 'TRACTCE'
    elif year <= 2018:
        svi_year = 2016
    else:
        svi_year = 2018
    
        
    svi_file = os.path.join(svi_dir, f'Massachusetts_SVI_{svi_year}.csv')
    svi_df = pd.read_csv(svi_file)
    
    if svi_year >= 2016:
        svi_df.loc[:,tract_col] = svi_df.FIPS.astype(str).apply(lambda x: x[5:])
    
    svi_df.loc[:,tract_col] = svi_df[tract_col].astype(str)
    svi_df.loc[:,geo_col] = svi_df[geo_col].astype(str)
    svi_df.loc[:,'ROUNDED_TRACT'] = svi_df[tract_col].astype(str).apply(lambda x: x[:-2] +'00' )
    
    target_cols = ['theme_1_pctile', 'theme_2_pctile', 'theme_3_pctile', 'theme_4_pctile', 'svi_pctile']
    svi_df = svi_df.rename(columns={theme:target for theme, target in zip(theme_cols, target_cols)})

    svi_df_rounded = svi_df.groupby('ROUNDED_TRACT').mean().reset_index()
    
    for month in range(1, 13):

        these_deaths = deaths_gdf[(deaths_gdf['year']==year)&(deaths_gdf['month']==month)]

        print(f'Deaths from {year} using SVI {svi_year}')

        first_merged_df = these_deaths.merge(svi_df[target_cols+[tract_col, geo_col]], left_on='GEOID', right_on=geo_col, how='left', indicator=True)
        if tract_col != 'TRACTCE':
            first_merged_df = first_merged_df.drop(columns=[tract_col])
        else:
            first_merged_df = first_merged_df.rename(columns={'TRACTCE_x':'TRACTCE'})
        unmerged_df = first_merged_df[first_merged_df['_merge']=='left_only'].drop(columns=target_cols+['_merge'])
        first_merged_df = first_merged_df[first_merged_df['_merge']=='both'].drop(columns=['_merge'])
        count_unmerged = len(unmerged_df)
        print(f"Successfully merged {(len(first_merged_df)-count_unmerged)/len(first_merged_df)*100:.1f}% on first pass")

        unmerged_df.loc[:,'ROUNDED_TRACTCE'] = unmerged_df['TRACTCE'].astype(str).apply(lambda x: x[:-2] +'00' ).astype(str)

        second_merged_df = unmerged_df.merge(svi_df_rounded[target_cols+['ROUNDED_TRACT']], left_on='ROUNDED_TRACTCE', right_on='ROUNDED_TRACT', how='left', indicator=True)
        unmerged_df = second_merged_df[second_merged_df['_merge']=='left_only'].drop(columns=target_cols+['_merge'])
        second_merged_df = second_merged_df.drop(columns=['_merge'])
        count_unmerged = len(unmerged_df)
        print(f"Successfully merged {(len(second_merged_df)-count_unmerged)/len(second_merged_df)*100:.1f}% of remaineder on second pass")

        merged_df = first_merged_df.append(second_merged_df)
        all_df = all_df.append(merged_df)


Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass


/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)
/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)
/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2000 using SVI 2000


/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)
/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)
/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2001 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2001 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2001 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass
Deaths from 2001 using SVI 2000
Successfully merged 47.9% on first pass
Successfully merged 92.6% of remaineder on second pass


...

/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)


Deaths from 2021 using SVI 2018
Successfully merged 77.6% on first pass
Successfully merged 83.5% of remaineder on second pass


/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)


Deaths from 2021 using SVI 2018
Successfully merged 77.6% on first pass
Successfully merged 83.5% of remaineder on second pass


/tmp/ipykernel_34672/1141560685.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = first_merged_df.append(second_merged_df)
/tmp/ipykernel_34672/1141560685.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_df = all_df.append(merged_df)


In [6]:
no_data = len(all_df[all_df['svi_pctile']==-999])
print(f'{no_data/len(all_df)*100:.1f}% of data is missing, replacing with mean')
all_df =all_df.replace(-999, np.nan)
all_df.loc[:,target_cols] = all_df[target_cols].fillna(all_df[target_cols].mean())

0.8% of data is missing, replacing with mean


In [7]:
svi_file = os.path.join(result_dir,'svi_month')
gpd.GeoDataFrame(all_df).to_file(svi_file)

/tmp/ipykernel_34672/3756858356.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(all_df).to_file(svi_file)


In [8]:
deaths_gdf['quarter']

KeyError: 'quarter'

In [89]:
all_df.set_index(['GEOID','year','quarter']).sort_index()

STATEFP COUNTYFP TRACTCE     NAME  \
GEOID       year quarter                                     
25001010100 2000 1            25      001   10100      101   
                 2            25      001   10100      101   
                 3            25      001   10100      101   
                 4            25      001   10100      101   
            2001 1            25      001   10100      101   
...                          ...      ...     ...      ...   
25027761402 2020 4            25      027  761402  7614.02   
            2021 1            25      027  761402  7614.02   
                 2            25      027  761402  7614.02   
                 3            25      027  761402  7614.02   
                 4            25      027  761402  7614.02   

                                      NAMELSAD  MTFCC FUNCSTAT     ALAND  \
GEOID       year quarter                                                   
25001010100 2000 1            Census Tract 101  G5020        S  25046216   
                 2            Census Tract 101  G5020        S  25046216   
                 3            Census Tract 101  G5020        S  25046216   
                 4            Census Tract 101  G5020        S  25046216   
            2001 1            Census Tract 101  G5020        S  25046216   
...                                        ...    ...      ...       ...   
25027761402 2020 4        Census Tract 7614.02  G5020        S  25568170   
            2021 1        Census Tract 7614.02  G5020        S  25568170   
                 2        Census Tract 7614.02  G5020        S  25568170   
                 3        Census Tract 7614.02  G5020        S  25568170   
                 4        Census Tract 7614.02  G5020        S  25568170   

                            AWATER     INTPTLAT  ...  \
GEOID       year quarter                         ...   
25001010100 2000 1        12765872  +42.0598291  ...   
                 2        12765872  +42.0598291  ...   
                 3        12765872  +42.0598291  ...   
                 4        12765872  +42.0598291  ...   
            2001 1        12765872  +42.0598291  ...   
...                            ...          ...  ...   
25027761402 2020 4          209089  +42.5313417  ...   
            2021 1          209089  +42.5313417  ...   
                 2          209089  +42.5313417  ...   
                 3          209089  +42.5313417  ...   
                 4          209089  +42.5313417  ...   

                                                                   geometry  \
GEOID       year quarter                                                      
25001010100 2000 1        POLYGON ((-70.25001 42.06410, -70.24959 42.065...   
                 2        POLYGON ((-70.25001 42.06410, -70.24959 42.065...   
                 3        POLYGON ((-70.25001 42.06410, -70.24959 42.065...   
                 4        POLYGON ((-70.25001 42.06410, -70.24959 42.065...   
            2001 1        POLYGON ((-70.25001 42.06410, -70.24959 42.065...   
...                                                                     ...   
25027761402 2020 4        POLYGON ((-71.63921 42.53096, -71.63906 42.531...   
            2021 1        POLYGON ((-71.63921 42.53096, -71.63906 42.531...   
                 2        POLYGON ((-71.63921 42.53096, -71.63906 42.531...   
                 3        POLYGON ((-71.63921 42.53096, -71.63906 42.531...   
                 4        POLYGON ((-71.63921 42.53096, -71.63906 42.531...   

                          theme_1_pctile  theme_2_pctile theme_3_pctile  \
GEOID       year quarter                                                  
25001010100 2000 1                0.7270          0.2972         0.5268   
                 2                0.7270          0.2972         0.5268   
                 3                0.7270          0.2972         0.5268   
                 4                0.7270          0.2972         0.5268   
            2001 1    

In [79]:
these_deaths.set_index(['GEOID','year','quarter']).sort_index().loc[pd.IndexSlice['25025040600',:,:],:]

,,,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,tract,deaths,geometry
GEOID,year,quarter,,,,,,,,,,,,,,
25025040600,2013,1,25,025,40600,406,Census Tract 406,G5020,S,1436234,413598,+42.3832248,-071.0698352,NaN,0.0,"POLYGON ((-71.08097 42.38232, -71.08094 42.382..."


In [73]:
first_merged_df[first_merged_df['TRACTCE']=='40600']

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,quarter,tract,deaths,geometry,theme_1_pctile,theme_2_pctile,theme_3_pctile,theme_4_pctile,svi_pctile,FIPS
217,25,025,40600,25025040600,406,Census Tract 406,G5020,S,1436234,413598,...,4,NaN,0.0,"POLYGON ((-71.08097 42.38232, -71.08094 42.382...",0.146,0.0588,0.4812,0.3325,0.172,25025040600
920,25,011,40600,25011040600,406,Census Tract 406,G5020,S,243617559,38074540,...,4,NaN,0.0,"POLYGON ((-72.53466 42.45847, -72.53426 42.459...",0.568,0.2994,0.3097,0.3789,0.356,25011040600


In [85]:
unmerged_df

,STATEFP,COUNTYFP,TRACTCE_x,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,year,quarter,tract,deaths,geometry,TRACTCE_y,FIPS
132,25,013,813001,25013813001,8130.01,Census Tract 8130.01,G5020,S,310046808,9613893,+42.1920235,-072.9640756,2013,1,NaN,0.0,"POLYGON ((-73.07505 42.10557, -73.07505 42.105...",NaN,NaN
133,25,013,802101,25013802101,8021.01,Census Tract 8021.01,G5020,S,4713082,345811,+42.0793013,-072.5687935,2013,1,NaN,0.0,"POLYGON ((-72.58935 42.07225, -72.58906 42.072...",NaN,NaN
134,25,013,813002,25013813002,8130.02,Census Tract 8130.02,G5020,S,193578178,3500767,+42.1311800,-072.8850695,2013,1,NaN,0.0,"POLYGON ((-72.98191 42.13669, -72.98118 42.136...",NaN,NaN
135,25,021,422402,25021422402,4224.02,Census Tract 4224.02,G5020,S,1990100,13733,+42.2152188,-070.9635618,2013,1,NaN,0.0,"POLYGON ((-70.97164 42.20039, -70.97133 42.202...",NaN,NaN
136,25,021,400201,25021400201,4002.01,Census Tract 4002.01,G5020,S,219587,0,+42.3438844,-071.1193879,2013,1,NaN,0.0,"POLYGON ((-71.12257 42.34411, -71.12244 42.344...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,25,009,208302,25009208302,2083.02,Census Tract 2083.02,G5020,S,4199517,722531,+42.4850384,-071.0040806,2013,1,NaN,0.0,"POLYGON ((-71.01723 42.48901, -71.01705 42.489...",NaN,NaN
1616,25,009,212101,25009212101,2121.01,Census Tract 2121.01,G5020,S,20571367,575025,+42.6136944,-071.0052311,2013,1,NaN,0.0,"POLYGON ((-71.05565 42.62288, -71.05509 42.625...",NaN,NaN
1617,25,009,208401,25009208401,2084.01,Census Tract 2084.01,G5020,S,6530788,386401,+42.4876121,-071.0307634,2013,1,NaN,0.0,"POLYGON ((-71.04695 42.48796, -71.04672 42.488...",NaN,NaN
1618,25,009,208402,25009208402,2084.02,Census Tract 2084.02,G5020,S,4410795,145126,+42.4662513,-071.0359714,2013,1,NaN,0.0,"POLYGON ((-71.05389 42.47878, -71.05373 42.479...",NaN,NaN


In [65]:
these_deaths[these_deaths['GEOID']=='25011040600']

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,year,quarter,tract,deaths,geometry
12260,25,011,40600,25011040600,406,Census Tract 406,G5020,S,243617559,38074540,+42.4603034,-072.3843581,2001,4,NaN,0.0,"POLYGON ((-72.53466 42.45847, -72.53426 42.459..."


In [80]:
unmerged_df

,STATEFP,COUNTYFP,TRACTCE_x,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,year,quarter,tract,deaths,geometry,TRACTCE_y,FIPS
132,25,013,813001,25013813001,8130.01,Census Tract 8130.01,G5020,S,310046808,9613893,+42.1920235,-072.9640756,2013,1,NaN,0.0,"POLYGON ((-73.07505 42.10557, -73.07505 42.105...",NaN,NaN
133,25,013,802101,25013802101,8021.01,Census Tract 8021.01,G5020,S,4713082,345811,+42.0793013,-072.5687935,2013,1,NaN,0.0,"POLYGON ((-72.58935 42.07225, -72.58906 42.072...",NaN,NaN
134,25,013,813002,25013813002,8130.02,Census Tract 8130.02,G5020,S,193578178,3500767,+42.1311800,-072.8850695,2013,1,NaN,0.0,"POLYGON ((-72.98191 42.13669, -72.98118 42.136...",NaN,NaN
135,25,021,422402,25021422402,4224.02,Census Tract 4224.02,G5020,S,1990100,13733,+42.2152188,-070.9635618,2013,1,NaN,0.0,"POLYGON ((-70.97164 42.20039, -70.97133 42.202...",NaN,NaN
136,25,021,400201,25021400201,4002.01,Census Tract 4002.01,G5020,S,219587,0,+42.3438844,-071.1193879,2013,1,NaN,0.0,"POLYGON ((-71.12257 42.34411, -71.12244 42.344...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,25,009,208302,25009208302,2083.02,Census Tract 2083.02,G5020,S,4199517,722531,+42.4850384,-071.0040806,2013,1,NaN,0.0,"POLYGON ((-71.01723 42.48901, -71.01705 42.489...",NaN,NaN
1616,25,009,212101,25009212101,2121.01,Census Tract 2121.01,G5020,S,20571367,575025,+42.6136944,-071.0052311,2013,1,NaN,0.0,"POLYGON ((-71.05565 42.62288, -71.05509 42.625...",NaN,NaN
1617,25,009,208401,25009208401,2084.01,Census Tract 2084.01,G5020,S,6530788,386401,+42.4876121,-071.0307634,2013,1,NaN,0.0,"POLYGON ((-71.04695 42.48796, -71.04672 42.488...",NaN,NaN
1618,25,009,208402,25009208402,2084.02,Census Tract 2084.02,G5020,S,4410795,145126,+42.4662513,-071.0359714,2013,1,NaN,0.0,"POLYGON ((-71.05389 42.47878, -71.05373 42.479...",NaN,NaN
